## Scatter plot for the different drone components

In [7]:
# This function tests if a component is dominated
# return 0 if non dominated, the number of domination other else
# inputs :
# x_,y_ : the  component's characteristics to test
# X_,Y_ : the  data set characteristics

def dominated(x_,y_,X_,Y_):
    compteur=0
    for a,b in zip(X_,Y_):
        # a>x_ for high torque and  b<y_ for small weight
        if (a>x_) and (b<y_):
            compteur +=1
    return compteur

### Automatic creation of pareto charts for motor, battery, ESC and propellers components

In [8]:
# %matplotlib inline
from ipywidgets import *
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd


In [9]:

path='./Motors/'
df_mot = pd.read_csv(path+'Motors_Data.csv', sep=';')

path='./ESC/'
df_esc = pd.read_csv(path+'ESC_data.csv', sep=';')

path='./Batteries/'
df_bat = pd.read_csv(path+'Batteries_Data.csv', sep=';')

path='./Propeller/'
df_pro = pd.read_csv(path+'Propeller_Data.csv', sep=';')


Mot_Par1=Dropdown(
    options= df_mot.columns.values,
    description='Motor parameter to maximize:',
    value='Tnom_Nm',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

Mot_Par2=Dropdown(
    options= df_mot.columns.values,
    description='Motor parameter to minimize:',
    value='weight_g',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

ESC_Par1=Dropdown(
    options= df_esc.columns.values,
    value='Pmax.in[W]',
    description='ESC parameter to maximize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

ESC_Par2=Dropdown(
    options= df_esc.columns.values,
    value='m[g]',
    description='ESC parameter to minimize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

Bat_Par1=Dropdown(
    options= df_bat.columns.values,
    value='Energy_J',
    description='Battery parameter to maximize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

Bat_Par2=Dropdown(
    options= df_bat.columns.values,
    value='Weight_g',
    description='Battery parameter to maximize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

Pro_Par1=Dropdown(
    options= df_pro.columns.values,
    value='Thrust',
    description='Propeller parameter to maximize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'})

Pro_Par2=Dropdown(
    options= df_pro.columns.values,
    value='DIAMETER',
    description='Propeller parameter to minimize:',
    disabled=False,style = {'description_width': '150pt'},layout = {'width': '300pt'}
)
button=widgets.Button(description='Plot')

box=VBox([HBox([Mot_Par1,Mot_Par2]),HBox([ESC_Par1,ESC_Par2]),HBox([Bat_Par1,Bat_Par2]),HBox([Pro_Par1,Pro_Par2])])
display(box)

In [4]:
def printing():
###### CODE ######  

        print('Loading')
        path='./Motors/'
        df_mot = pd.read_csv(path+'Motors_Data.csv', sep=';')

        path='./ESC/'
        df_esc = pd.read_csv(path+'ESC_data.csv', sep=';')

        path='./Batteries/'
        df_bat = pd.read_csv(path+'Batteries_Data.csv', sep=';')

        path='./Propeller/'
        df_pro = pd.read_csv(path+'Propeller_Data.csv', sep=';')


        df_mot['Dominated']=0

        for row in range(len(df_mot[Mot_Par1.value])):
            if dominated(df_mot.loc[row,Mot_Par1.value], df_mot.loc[row,Mot_Par2.value],df_mot[Mot_Par1.value].values,df_mot[Mot_Par2.value].values)>0:
                df_mot.loc[row,'Dominated']=1    

        #     df_mot[df_mot['Dominated']==0].to_csv(r'./Motors/Non-Dominated-Motors.csv',sep=';')#We save the non-dominated series to process after.

        color_wheel = {0: 'r', 
                       1: 'b'} #we color in red the non-dominated series

        colorsM = df_mot['Dominated'].map(lambda x: color_wheel.get(x))

        pd.plotting.scatter_matrix(df_mot[['Tnom_Nm','weight_g','Kt_Nm_A']], color=colorsM, figsize=[15,10], s=200)

        plt.suptitle('Motor')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_mot[df_mot['Dominated']==0],title='"Download CSV file pareto Motors', filename='motors_pareto.csv'))
 
        #     plt.savefig('ScatterMotor.pdf',dpi=1200,format='pdf')

        ###### ESC #######
        df_esc['Dominated']=0
        df_esc = df_esc.reset_index(drop=True)# clear row index after ignoring one family battery

        for row in range(len(df_esc['Pmax.in[W]'])):
            if dominated(df_esc.loc[row,ESC_Par1.value], df_esc.loc[row,ESC_Par2.value],df_esc[ESC_Par1.value].values,df_esc[ESC_Par2.value].values)>0:
                df_esc.loc[row,'Dominated']=1    

        #     df_esc[df_esc['Dominated']==0].to_csv(r'./ESC/Non-Dominated-ESC.csv',sep=';')#We save the non-dominated series to process after.

        color_wheel = {0: 'r', 
                       1: 'b'}

        colorsE = df_esc['Dominated'].map(lambda x: color_wheel.get(x))

        df_esc=df_esc.loc[:, df_esc.columns != 'Dominated']

        pd.plotting.scatter_matrix(df_esc, color=colorsE, figsize=[15,10], s=200)
        plt.suptitle('ESC')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})    
        display(create_download_link(df_esc,title='"Download CSV file pareto ESC', filename='esc_pareto.csv'))


        ##### battery ######
        df_bat['Dominated']=0

        df_bat = df_bat.reset_index(drop=True)# clear row index after ignoring one family battery

        for row in range(len(df_bat['Weight_g'])):
            if dominated(df_bat.loc[row,Bat_Par1.value], df_bat.loc[row,Bat_Par2.value],df_bat[Bat_Par1.value].values,df_bat[Bat_Par2.value].values)>0:
                df_bat.loc[row,'Dominated']=1    

        #     df_bat[df_bat['Dominated']==0].to_csv(r'./Batteries/Non-Dominated-Batteries.csv',sep=';')#We save the non-dominated series to process after.

        color_wheel = {0: 'r', 
                       1: 'b'}

        colorsB = df_bat['Dominated'].map(lambda x: color_wheel.get(x))

        pd.plotting.scatter_matrix(df_bat[['Energy_J','Capacity_mAh','Weight_g','Voltage_V','Energy_density_Wh_kg']], color=colorsB, figsize=[15,10], s=200);
        plt.suptitle('Battery')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_bat[df_bat['Dominated']==0],title='"Download CSV file pareto batteries', filename='batteries_pareto.csv'))

        ##### propeller ####

        df_pro['Dominated']=0
        df_pro = df_pro.reset_index(drop=True)# clear row index after ignoring one family battery

        for row in range(len(df_pro['Ct'])):
            if dominated(df_pro.loc[row,Pro_Par1.value], df_pro.loc[row,Pro_Par2.value],df_pro[Pro_Par1.value].values,df_pro[Pro_Par2.value].values)>0:
                df_pro.loc[row,'Dominated']=1    

        color_wheel = {0: 'r',
                       1: 'b'}

        linewidth = {0: 300,
                       1: 100}


        colorsP = df_pro['Dominated'].map(lambda x: color_wheel.get(x))
        #     s = df_pro['Dominated'].map(lambda x: linewidth.get(x))

        #     df_pro[df_pro['Dominated']==0].to_csv(r'./Propeller/Non-Dominated-Propeller.csv',sep=';')#We save the non-dominated series to process after.
        pd.plotting.scatter_matrix(df_pro[['BETA','DIAMETER','Ct','Cp']], color=colorsP, figsize=[15,10], s=200)
        plt.suptitle('Propeller')
        plt.tick_params(axis = 'both', labelsize = 14)
        plt.rcParams.update({'font.size': 14})
        display(create_download_link(df_pro[df_pro['Dominated']==0],title='"Download CSV file pareto propellers', filename='propellers_pareto.csv'))

#         plt.close('all')

        plt.show()
###### CODE ######   


In [5]:
# %matplotlib widget
# To prevent automatic figure display when execution of the cell ends
%config InlineBackend.close_figures=False 

import matplotlib.pyplot as plt
import numpy as np

import base64
from IPython.display import HTML

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display,clear_output


In [6]:

# plt.ioff()
# ax=plt.gca()

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


button = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)

display(button)



output = widgets.Output()

@output.capture()


def on_button_clicked(b):
    plt.close("all")
    clear_output()
    printing()

# Mot_Par2.observe(on_button_clicked, Mot_Par2.value)
# ESC_Par1.observe(on_button_clicked, ESC_Par1.value)
# ESC_Par2.observe(on_button_clicked, Mot_Par2.value)
# Mot_Par1.observe(on_button_clicked, Mot_Par1.value)
# Mot_Par1.observe(on_button_clicked, Mot_Par1.value)
# Mot_Par1.observe(on_button_clicked, Mot_Par1.value)

button.on_click(on_button_clicked)
# Mot_Par1.observe(on_button_clicked, Mot_Par1.value)
display(output)

 




Button(button_style='info', description='Plot', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()